##Get GNSS values for lake survey

This guide helps to obtain the WSE for lakes when using Septrentrio GNSS
Steps:
Pre steps
0.0. Extract the Septentrio receivers' file .SBF
0.1. Prepare the file with the field data format
1. Convert native GNSS data to the Receiver Independent Exchange Format (RINEX) file type.
1.2. For the conversion use use SBF Converter, which is part of the Septentrio RxTools software package, which is freely available via the septentrio website. 
2. RINEX file is submitted for postprocessing using PPP software: https://webapp.geod.nrcan.gc.ca/geod/tools-outils/ppp.php.
3. Once downloading the files, use this script to:
3.1. Read the single or multiple csv files generated by the ppp software
3.2. If single file, only filter data in the valid time
3.3. If multiple file, unify the data and filter data in the valid time
3.4. Get the orthometric height using the software 
     NOTE: Specify the Geoid model used for this calculation. A useful resourse https://observablehq.com/@cehanagan/geoid-height-calculator#understanding-the-geoid-calculation
3.5. Get the median height of the Antenna (mH) and the standard deviation of the antenna heights (stdH)
3.6. Subtract the median height from the antenna to the water surface (mHA) from the orthometric height. NOTE: I'm using the median, but the mean is also used
3.7. Calculate the standard deviation of the antenna height (stdA)
3.8. Calculate the mean error (stdH+stdA+0.01 m). where 0.01 is the absolute error of the antenna heights
3.9. Prepare output results

In [ ]:
two sided
Siempre
m1<=m2
less
m1>=m2
greater

pvalue>alpha
 conclusion='Significative equal'
else
 conclusion='Significantly different'

t,x=